<a href="https://colab.research.google.com/github/soham-malhotra/Library_Database_Project/blob/main/DatabaseProjectLibrary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Management System

---

### Introduction
This project aims to create a database-driven Library Management System (LMS) to manage books, borrowers, and transactions efficiently. The database will allow librarians to track the availability of books, manage borrower details, and log the issuance and return of books.

### Goals
To develop an easy-to-manage database system to keep track of:

*   ***Books*** (title, author, genre, availability, etc.)
*   ***Borrowers*** (name, contact info, borrowed books, etc.)
*   ***Transactions*** (check-out, return, overdue fines)


### What were the key relationships we focused on


*  A borrower can borrow multiple books unless he has an overdue book
*  A book can be borrowed by one borrower at a time
*  If the borrower has the book for more than 15 days, and someone requests it,
he must return it
*  Each transaction logs a book’s borrowing and returning.

---

##Imports





















In [130]:
# for the creation and operation of the database
import os
import polars as pl

#for the interactivity
import ipywidgets as widgets
from IPython.display import display



  We add the necessary imports to create the database, for which we use polars and this database is then serialized and stored/accessed using parquet files, When you write a DataFrame to a Parquet file, the data is serialized into a binary format. This means that the information is converted into a compact representation that can be easily written to disk.


  Now we will import the initial library database, which we created as a csv file and which will be manipulated using polars


In [131]:
#link to the csv file for the library books
url_books = 'https://raw.githubusercontent.com/soham-malhotra/Library_Database_Project/refs/heads/main/library.csv'
url_users = 'https://raw.githubusercontent.com/soham-malhotra/Library_Database_Project/refs/heads/main/users.csv'

#get the dataframes
df_books = pl.read_csv(url_books)
df_users = pl.read_csv(url_users)

#print the first 4 rows of the books database and the first 2 rows of the users database
print(df_books.head(6))
print(df_users.head(4))


shape: (6, 5)
┌─────────┬─────────────────────┬───────────────────────┬────────────────┬───────────────────┐
│ User ID ┆ Author Name         ┆ Book Title            ┆ ISBN           ┆ Quantity in Stock │
│ ---     ┆ ---                 ┆ ---                   ┆ ---            ┆ ---               │
│ i64     ┆ str                 ┆ str                   ┆ str            ┆ i64               │
╞═════════╪═════════════════════╪═══════════════════════╪════════════════╪═══════════════════╡
│ 1       ┆ George Orwell       ┆ 1984                  ┆ 978-0451524935 ┆ 3                 │
│ 2       ┆ J.K. Rowling        ┆ Harry Potter and the  ┆ 978-0439708180 ┆ 5                 │
│         ┆                     ┆ Sorcerer'…            ┆                ┆                   │
│ 3       ┆ F. Scott Fitzgerald ┆ The Great Gatsby      ┆ 978-0743273565 ┆ 2                 │
│ 4       ┆ Harper Lee          ┆ To Kill a Mockingbird ┆ 978-0061120084 ┆ 4                 │
│ 5       ┆ J.R.R. Tolkien      ┆ Th

As we can see the database is clearly visible and contents clearly displayed. Now that this has been created, we must create Classes to define the system and be able to manipulate it. Note that Polars was used as it is more efficent than pandas, albeit as the database is so small the difference in speed is negligible

In [132]:
# Define the Book class
class Book:
    # initialise Book with all necessary components
    def __init__(self, title, author, isbn):
        self.title = title
        self.author = author
        self.isbn = isbn

    # print Book info (e.g. Harry Potter The Goblet of Fire by JK Rowling, ISBN: 0-7475-5079-4)
    def __repr__(self):
      return f"{self.title} by {self.author}, ISBN : {self.isbn}"

class User:

    # 5 as there are already 5 users in the database
    uID_counter = 10

    #initialise User with all necessary components
    def __init__(self, username, email):
      self.username = username
      self.borrowed_books = "None"
      self.email = email
      User.uID_counter += 1
      self.userid = User.uID_counter

    #print user info
    def __repr__(self):
      return f"\n  Username               :{self.username}\n  Contact Information    :{self.email}\n  Borrowed Books         :{self.borrowed_books}\n\n"


Now we have defined the Book class, Borrower class with all their required components. Nowe we can create the Library class which is the main component of the project, with functions on database manipulation and usage using these classes


In [133]:
class Library:
    #initialise all necessary components
    def __init__(self, filename_books="library_books.parquet", filename_users="library_users.parquet"):
        self.filename_books = filename_books
        self.filename_users = filename_users
        df_books.write_parquet(self.filename_books)
        df_users.write_parquet(self.filename_users)
        self.books_df = self.load_database(filename_books) # Polars Dataframe
        self.users_df = self.load_database(filename_users) # Polars Dataframe


    # Load books from file
    def load_database(self, filename):
       if os.path.exists(filename):
          return pl.read_parquet(filename)
       else:
          return pl.DataFrame()


    # updated database, option is either "books" or "users"
    def update_database(self, option):
      if option == "books":
        self.books_df.write_parquet(self.filename_books)
      elif option == "users":
        self.users_df.write_parquet(self.filename_users)
      else:
        print(f"Sorry, your Option: {option} is not a valid one")
      print("Database update complete")


    # Get all books in database
    def view_all(self):
      if self.books_df.height == 0:
        print("ALL THE BOOKS ARE GONE !")
      else:
        print(self.books_df)

    # Creates a user
    def create_user(self, email, username):
      filtered_df = self.users_df.filter(pl.col("username") == username)
      if filtered_df.height > 0:
        print("choose another username")
      elif self.users_df.filter(pl.col("email") == email).height > 0 :
        actual_username = filtered_df.select("username")
        print(f"this email is already in the system, your username is {actual_username}")
      else:
        new_user = User(username, email)
        print("Created new user:", new_user)

        new_user_data = {
           "username": [new_user.username],
           "User ID": [new_user.userid],
           "email": [new_user.email],
           "Borrowed Books": [new_user.borrowed_books]
        }

        # Convert the new user data into a Polars DataFrame
        new_user_df = pl.DataFrame(new_user_data)

        # Append the new user to the existing DataFrame
        self.users_df = pl.concat([self.users_df, new_user_df], how="vertical")
        ##print(f"User '{username}' added successfully.")
        #print(self.users_df)
        print(self.users_df)
        self.update_database("books")

    # returns user or creates it if not there option can only be "username" or "email" !
    def get_user(self, option, value):
      if option == "username" or option == "email":
        filtered_df = self.users_df.filter(pl.col(option) == value)
        if filtered_df.height > 0:
          return filtered_df.select(option)
        else:
          print("a user with these credentials doesnt exist")
          return
      else:
        print("incorrect option provided, need either username or email")
        return

     # check if book exists / if its in stock using "ISBN" or "Book Title"
    def book_exists(self, option, value):
     if option == "ISBN" or option == "Book Title":
        filtered_df = self.books_df.filter(pl.col(option) == value)
        if filtered_df.height == 1 and filtered_df.select("Quantity in Stock").to_numpy()[0][0] != 0:
          print("Yes the book is available !")
          print(filtered_df)
          return True
        else:
          print("Book doesnt exist")
          return False
     else:
      print(f"option input {option} not valid")
      return False

    # return a database of all the author's works
    def get_all_author_books(self, author_name):
      filtered_df = self.books_df.filter(pl.col("Author Name") == author_name)
      if filtered_df.height > 0:
        print(filtered_df)
      else:
        print("Im afraid we dont have this author in our humble library")


    # Add a book (in case of donation or return for example)
    def add_book(self, username, book_title, book_author, book_isbn):
        # Check if the book already exists
        if self.books_df.filter(pl.col("ISBN") == book_isbn).height > 0:
            # If it exists, increase the quantity in stock
            self.books_df = self.books_df.with_columns(
                pl.when(pl.col("ISBN") == book_isbn)
                .then(pl.col("Quantity in Stock") + 1)
                .otherwise(pl.col("Quantity in Stock"))
                .alias("Quantity in Stock")
            )
            print("Return or Donation Successful, Thank you !")
        else:
            # If it doesn't exist, create a new book entry
            new_book = Book(book_title, book_author, book_isbn)
            user_id = user_id = (self.users_df.filter(pl.col("username") == username).select("User ID").item()
                      if self.users_df.filter(pl.col("username") == username).height > 0 else print("User Doesnt Exist !"))

            new_book_df = pl.DataFrame([{
                'User ID' : user_id,
                'Author Name': new_book.author,
                'Book Title': new_book.title,
                'ISBN': new_book.isbn,
                'Quantity in Stock': 1
            }])
            self.books_df = pl.concat([self.books_df, new_book_df], how="vertical")
            print(f"New book '{new_book.title}' by {new_book.author} added to the library.")
            print("Thank you for your donation !")
        self.update_database("books")

    # borrow book from library
    def borrow_book(self, borrower_username, option, value):
        filtered_user = self.users_df.filter(pl.col("username") == borrower_username)
        if filtered_user.height == 0:
            print("User does not exist.")
            return

        filtered_book = self.books_df.filter(pl.col(option) == value)
        if filtered_book.height > 0 and filtered_book.select("Quantity in Stock").to_numpy()[0][0] > 0:
            # Update the book quantity
            self.books_df = self.books_df.with_columns(
                pl.when(pl.col(option) == value)
                .then(pl.col("Quantity in Stock") - 1)
                .otherwise(pl.col("Quantity in Stock"))
                .alias("Quantity in Stock")
            )

            # Add book to user's borrowed books
            user = filtered_user.to_dicts()[0]
            user_id = user['User ID']
            print("user: ", user, "and the user_id ", user_id)
            self.users_df = self.users_df.with_columns(
                pl.when(pl.col("User ID") == user_id)
                .then(
                    pl.when(pl.col("Borrowed Books").str.strip_chars() == "None")  # Check if Borrowed Books is empty
                    .then(pl.lit(value))  # If empty, set the new book
                    .otherwise(pl.col("Borrowed Books") + pl.lit(", ") + pl.lit(value)))
                .otherwise(pl.col("Borrowed Books"))
                .alias("Borrowed Books")
            )

            print(self.users_df)

            #    .then(pl.col("Borrowed Books")
            #    .otherwise(pl.col("Borrowed Books"))
            #    .alias("Borrowed Books"))

            print(f"User '{borrower_username}' has borrowed the book '{value}'.")
            self.update_database("books")
            self.update_database("users")
        else:
            print("The book is not available for borrowing.")


Now that the library class is done, and we have all the necessary functions to manipulate the database, we create an interactive interface for the user to view, ask and update with the library, but first lets test the functionality

---

#Testing

In [134]:
# Create a library instance
library = Library()

Create the library

In [135]:
#Test create user functionality
library.create_user("abcd@gmail","abcd")

Created new user: 
  Username               :abcd
  Contact Information    :abcd@gmail
  Borrowed Books         :None


shape: (6, 4)
┌──────────┬─────────┬───────────────────┬────────────────┐
│ username ┆ User ID ┆ email             ┆ Borrowed Books │
│ ---      ┆ ---     ┆ ---               ┆ ---            │
│ str      ┆ i64     ┆ str               ┆ str            │
╞══════════╪═════════╪═══════════════════╪════════════════╡
│ user1    ┆ 1       ┆ alice@gmail.com   ┆ None           │
│ user2    ┆ 2       ┆ bob@gmail.com     ┆ None           │
│ user3    ┆ 3       ┆ charlie@gmail.com ┆ None           │
│ user4    ┆ 4       ┆ david@gmail.com   ┆ None           │
│ user5    ┆ 5       ┆ eve@gmail.com     ┆ None           │
│ abcd     ┆ 11      ┆ abcd@gmail        ┆ None           │
└──────────┴─────────┴───────────────────┴────────────────┘
Database update complete


In [136]:
#Test add book functionality
library.add_book("user2","haha","omar","1111")
library.view_all()

New book 'haha' by omar added to the library.
Thank you for your donation !
Database update complete
shape: (11, 5)
┌─────────┬─────────────────────┬───────────────────────┬────────────────┬───────────────────┐
│ User ID ┆ Author Name         ┆ Book Title            ┆ ISBN           ┆ Quantity in Stock │
│ ---     ┆ ---                 ┆ ---                   ┆ ---            ┆ ---               │
│ i64     ┆ str                 ┆ str                   ┆ str            ┆ i64               │
╞═════════╪═════════════════════╪═══════════════════════╪════════════════╪═══════════════════╡
│ 1       ┆ George Orwell       ┆ 1984                  ┆ 978-0451524935 ┆ 3                 │
│ 2       ┆ J.K. Rowling        ┆ Harry Potter and the  ┆ 978-0439708180 ┆ 5                 │
│         ┆                     ┆ Sorcerer'…            ┆                ┆                   │
│ 3       ┆ F. Scott Fitzgerald ┆ The Great Gatsby      ┆ 978-0743273565 ┆ 2                 │
│ 4       ┆ Harper Lee       

In [137]:
#Test that user can borrow a book that was added
library.borrow_book("abcd","ISBN","1111")
library.view_all()

user:  {'username': 'abcd', 'User ID': 11, 'email': 'abcd@gmail', 'Borrowed Books': 'None'} and the user_id  11
shape: (6, 4)
┌──────────┬─────────┬───────────────────┬────────────────┐
│ username ┆ User ID ┆ email             ┆ Borrowed Books │
│ ---      ┆ ---     ┆ ---               ┆ ---            │
│ str      ┆ i64     ┆ str               ┆ str            │
╞══════════╪═════════╪═══════════════════╪════════════════╡
│ user1    ┆ 1       ┆ alice@gmail.com   ┆ None           │
│ user2    ┆ 2       ┆ bob@gmail.com     ┆ None           │
│ user3    ┆ 3       ┆ charlie@gmail.com ┆ None           │
│ user4    ┆ 4       ┆ david@gmail.com   ┆ None           │
│ user5    ┆ 5       ┆ eve@gmail.com     ┆ None           │
│ abcd     ┆ 11      ┆ abcd@gmail        ┆ 1111           │
└──────────┴─────────┴───────────────────┴────────────────┘
User 'abcd' has borrowed the book '1111'.
Database update complete
Database update complete
shape: (11, 5)
┌─────────┬─────────────────────┬──────────────

In [138]:
#Test that if you borrow a quantity 0 book it gives an error
library.borrow_book("user3","ISBN","1111")

The book is not available for borrowing.




---



# **THE LIBRARY**


In [139]:
# Create widgets for library actions
username_input = widgets.Text(description="Username:")
email_input = widgets.Text(description="Email:")
book_title_input = widgets.Text(description="Book Title:")
book_author_input = widgets.Text(description="Author:")
book_isbn_input = widgets.Text(description="ISBN:")
option_input_borrow = widgets.Dropdown(description="", options=["ISBN","Book Title"])
option_input_search = widgets.Dropdown(description="", options=["ISBN","Book Title"])
value_input_borrow = widgets.Text(description="Borrow:")
value_input_search = widgets.Text(description="Search for:")
action_output = widgets.Output()


# Create buttons for actions
create_user_button = widgets.Button(description="Create User")
view_books_button = widgets.Button(description="View All Books")
add_book_button = widgets.Button(description="Add Book")
borrow_book_button = widgets.Button(description="Borrow Book")
search_book_button = widgets.Button(description="Search Book")


# Define button click events
def create_user(button):
    with action_output:
        action_output.clear_output()
        library.create_user(email_input.value, username_input.value)

def view_books(button):
    with action_output:
        action_output.clear_output()
        library.view_all()

def add_book(button):
    with action_output:
        action_output.clear_output()
        library.add_book(username_input.value, book_title_input.value, book_author_input.value, book_isbn_input.value)

def borrow_book(button):
    with action_output:
        action_output.clear_output()
        #if library.book_exists(option_input_borrow.value, value_input_borrow.value):
        library.borrow_book(username_input.value, option_input_borrow.value, value_input_borrow.value)
        #  print("Thank you for borrowing !")

def search_book(button):
  with action_output:
    action_output.clear_output()
    library.book_exists(option_input_search.value, value_input_search.value)


    def book_exists(self, option, value):
     if option == "ISBN" or option == "Book Title":
        filtered_df = self.books_df.filter(pl.col(option) == value)
        if filtered_df.height == 1 and filtered_df.select("Quantity in Stock").to_numpy()[0][0] != 0:
          print("Yes the book is available !")
          print(filtered_df)
          return True
        else:
          print("Book doesnt exist")
          return False
     else:
      print(f"option input {option} not valid")
      return False


# Attach event handlers
create_user_button.on_click(create_user)
view_books_button.on_click(view_books)
add_book_button.on_click(add_book)
borrow_book_button.on_click(borrow_book)
search_book_button.on_click(search_book)


# Layout the widgets
print(" "*20 + "The Library" + " "*20)
display(username_input, email_input, create_user_button)
print("*"*60)
display(book_title_input, book_author_input, book_isbn_input,add_book_button)
print("*"*60)
display(option_input_search, value_input_search, search_book_button)
print("*"*60)
display(option_input_borrow, value_input_borrow, borrow_book_button)
print("*"*60)
display(view_books_button, action_output)

                    The Library                    


Text(value='', description='Username:')

Text(value='', description='Email:')

Button(description='Create User', style=ButtonStyle())

************************************************************


...

Text(value='', description='Borrow:')

Button(description='Borrow Book', style=ButtonStyle())

************************************************************


Button(description='View All Books', style=ButtonStyle())

Output()

Some more experimental Interface

In [140]:
library = Library()

#create widgets
action_output = widgets.Output()
login_button = widgets.Button(description="Login")
username_input = widgets.Text(description = "Username")
email_input = widgets.Text(description = "Email")



#login button click event
def login_user(button):
    with action_output:
        action_output.clear_output()
        print("-"*60)
        if(library.get_user("username", username_input.value)):
            login_button.layout.display = 'none'
            username_input.layout.display = 'none'
            email_input.layout.display = 'none'
            print("Welcome back !")
            print("-"*60)
        else:
          login_button.layout.display = 'none'
          username_input.layout.display = 'none'
          email_input.layout.display = 'none'
          print("Creating an account...")
          library.create_user(email_input.value, username_input.value)
          print("-"*60)


login_button.on_click(login_user)


# Layout the widgets
print(" "*20 + "The Library" + " "*20)
display(username_input, email_input, login_button)
display(action_output)

                    The Library                    


Text(value='', description='Username')

Text(value='', description='Email')

Button(description='Login', style=ButtonStyle())

Output()